# ANALISIS MULTITEMPORAL MAATE

El presente Notebook tiene una serie de scripts que se pueden utilizar para seguir con la misma metodología para el análisis de la variación de uso de suelo. Existen dos fuentes de información fundamentales: La capa de cobertura de uso de suelo Nacional proporcionada por el MAATE en el SUIA (Sistema Único de Información Ambiental) y la división geopolítica. Esta información se analiza en capas de formato shapefile.

In [6]:
!pip install geopandas rtree mapclassify 
!pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]



In [ ]:
# ==============================================================================
# SCRIPT PARA ANÁLISIS DE COBERTURA DE SUELO (VERSIÓN LOCAL)
# ==============================================================================
# Este script realiza los siguientes pasos:
# 1. Itera sobre cada shapefile de área de estudio en una carpeta específica.
# 2. Para cada área, corta tres capas de cobertura de suelo (2018, 2020, 2022).
# 3. Realiza un análisis de superposición para asignar la provincia, cantón y
#    parroquia a cada polígono de cobertura cortado.
#    - Criterio de asignación: La unidad geopolítica con la mayor área de superposición.
# 4. Calcula el área en hectáreas (ha) para cada polígono de cobertura.
# 5. Estandariza la tabla de atributos con las columnas requeridas.
# 6. Guarda los resultados en shapefiles separados para cada combinación
#    de área de estudio y año de cobertura.
# ==============================================================================

# ------------------------------------------------------------------------------
# 1. IMPORTACIÓN DE LIBRERÍAS
# ------------------------------------------------------------------------------
# Asegúrate de haber instalado las librerías ejecutando en tu terminal:
# pip install geopandas rtree mapclassify

print("Importando librerías necesarias...")
import geopandas as gpd
import os
import warnings

# Ignorar advertencias futuras que puedan surgir de geopandas para una salida más limpia
warnings.simplefilter(action='ignore', category=FutureWarning)

print("Librerías importadas correctamente. ✅")

# ------------------------------------------------------------------------------
# 2. DEFINICIÓN DE RUTAS (PARA EJECUCIÓN LOCAL)
# ------------------------------------------------------------------------------
# ¡IMPORTANTE! Reemplaza estas rutas de ejemplo con las rutas reales de tu computadora.
# El uso de os.path.join asegura que las rutas funcionen en cualquier sistema operativo.

# --- EJEMPLO PARA WINDOWS ---
# Define la ruta a tu carpeta principal del proyecto.
# ruta_base = "C:/Users/TuUsuario/Documents/GIZ/ESTUDIO_GEODATA"

# --- EJEMPLO PARA MACOS O LINUX ---
# ruta_base = "/home/tu_usuario/documentos/GIZ/ESTUDIO_GEODATA"

# **MODIFICA ESTA LÍNEA CON LA RUTA A TU CARPETA PRINCIPAL**
ruta_base = "C:/Users/nacho/Documents/Github/GIZ" # <-- ¡¡¡CAMBIA ESTO!!!

# El script construye las demás rutas automáticamente a partir de la ruta base.
# Asegúrate de que tus subcarpetas se llamen igual que en el ejemplo.
ruta_areas_estudio = os.path.join(ruta_base, "area_estudio")
ruta_coberturas = os.path.join(ruta_base, "capas_cobertura/napo_sucumbios_orellana")
ruta_geopolitica = os.path.join(ruta_base, "division_geopolitica")
ruta_resultados = os.path.join(ruta_base, "results")

# Crear la carpeta de resultados si no existe.
if not os.path.exists(ruta_resultados):
    os.makedirs(ruta_resultados)
    print(f"Carpeta de resultados creada en: {ruta_resultados}")

# ------------------------------------------------------------------------------
# 3. CONFIGURACIÓN DE CAPAS DE COBERTURA (Sin cambios)
# ------------------------------------------------------------------------------
coberturas_info = [
    {
        'year': '2018',
        'file': 'COB2018_SOLVED_cortado.shp',
        'col': 'cobertura0'
    },
    {
        'year': '2020',
        'file': 'COB2020_SOLVED_cortado.shp',
        'col': 'ctn2'
    },
    {
        'year': '2022',
        'file': 'COB2022_SOLVED_cortado.shp',
        'col': 'ctn2'
    }
]

# ------------------------------------------------------------------------------
# 4. CARGA DE CAPAS GEOPOLÍTICAS (Sin cambios)
# ------------------------------------------------------------------------------
try:
    print("\nCargando capas de división geopolítica...")
    provincias_gdf = gpd.read_file(os.path.join(ruta_geopolitica, 'provincial.shp'))
    cantones_gdf = gpd.read_file(os.path.join(ruta_geopolitica, 'cantonal.shp'))
    parroquias_gdf = gpd.read_file(os.path.join(ruta_geopolitica, 'parroquial.shp'))
    print("Capas geopolíticas cargadas exitosamente. 🗺️")
except Exception as e:
    print(f"Error al cargar las capas geopolíticas: {e}")
    exit()

col_provincia = 'DPA_DESPRO'
col_canton = 'DPA_DESCAN'
col_parroquia = 'DPA_DESPAR'

# ------------------------------------------------------------------------------
# 5. FUNCIÓN PARA ASIGNACIÓN GEOPOLÍTICA (Sin cambios)
# ------------------------------------------------------------------------------
def asignar_unidad_geopolitica(datos_cortados, capa_geopolitica, nombre_columna):
    datos_cortados['temp_id'] = range(len(datos_cortados))
    interseccion = gpd.overlay(datos_cortados, capa_geopolitica, how='intersection')
    interseccion['area_interseccion'] = interseccion.geometry.area
    mayor_superposicion = interseccion.sort_values('area_interseccion', ascending=False)
    mayor_superposicion = mayor_superposicion.drop_duplicates('temp_id')
    resultado = mayor_superposicion[['temp_id', nombre_columna]]
    datos_con_geo = datos_cortados.merge(resultado, on='temp_id', how='left')
    datos_con_geo = datos_con_geo.drop(columns=['temp_id'])
    return datos_con_geo

# ------------------------------------------------------------------------------
# 6. PROCESAMIENTO PRINCIPAL (Sin cambios)
# ------------------------------------------------------------------------------
print("\n--- INICIANDO PROCESO DE ANÁLISIS ESPACIAL ---")

for area_shp_nombre in os.listdir(ruta_areas_estudio):
    if area_shp_nombre.endswith('.shp'):
        nombre_base_area = os.path.splitext(area_shp_nombre)[0]
        ruta_area_completa = os.path.join(ruta_areas_estudio, area_shp_nombre)

        print(f"\n==================================================")
        print(f"Procesando Área de Estudio: {nombre_base_area}")
        print(f"==================================================")

        try:
            area_gdf = gpd.read_file(ruta_area_completa)
        except Exception as e:
            print(f"   -> Error al leer el archivo {area_shp_nombre}: {e}")
            continue

        for cob in coberturas_info:
            año = cob['year']
            archivo_cob = cob['file']
            columna_categoria = cob['col']
            ruta_cob_completa = os.path.join(ruta_coberturas, archivo_cob)

            print(f"\n   Procesando cobertura del año: {año}")

            if not os.path.exists(ruta_cob_completa):
                print(f"   -> ¡ADVERTENCIA! No se encontró el archivo: {archivo_cob}. Saltando...")
                continue

            try:
                cobertura_gdf = gpd.read_file(ruta_cob_completa)

                print("     1/5: Cortando capa de uso de suelo...")
                if area_gdf.crs != cobertura_gdf.crs:
                    print(f"       -> Advirtiendo: CRS no coincide. Reproyectando área de estudio al CRS de cobertura ({cobertura_gdf.crs.to_string()})")
                    area_gdf = area_gdf.to_crs(cobertura_gdf.crs)

                cortado_gdf = gpd.clip(cobertura_gdf, area_gdf)

                if cortado_gdf.empty:
                    print("     -> El área de estudio no se superpone con la capa de cobertura. Saltando.")
                    continue

                print("     2/5: Calculando área en hectáreas...")
                cortado_gdf['ha'] = cortado_gdf.geometry.area / 10000

                print("     3/5: Asignando información geopolítica...")
                resultado_con_prov = asignar_unidad_geopolitica(cortado_gdf, provincias_gdf, col_provincia)
                resultado_con_can = asignar_unidad_geopolitica(resultado_con_prov, cantones_gdf, col_canton)
                resultado_final = asignar_unidad_geopolitica(resultado_con_can, parroquias_gdf, col_parroquia)

                print("     4/5: Estandarizando tabla de atributos...")
                resultado_final = resultado_final.rename(columns={
                    columna_categoria: 'uso_suelo',
                    col_provincia: 'provincia',
                    col_canton: 'canton',
                    col_parroquia: 'parroquia'
                })

                columnas_deseadas = ['provincia', 'canton', 'parroquia', 'uso_suelo', 'ha', 'geometry']
                output_gdf = resultado_final[columnas_deseadas]

                for col in ['provincia', 'canton', 'parroquia']:
                    output_gdf[col] = output_gdf[col].fillna('INDEFINIDO')

                nombre_salida = f"cob{año}_{nombre_base_area}.shp"
                ruta_salida_completa = os.path.join(ruta_resultados, nombre_salida)
                print(f"     5/5: Guardando resultado en: {nombre_salida}")

                output_gdf.to_file(ruta_salida_completa, driver='ESRI Shapefile', encoding='utf-8')
                print(f"     -> Archivo guardado con éxito. ✨")

            except Exception as e:
                print(f"   -> Ocurrió un error inesperado al procesar {archivo_cob}: {e}")

print("\n--- PROCESO FINALIZADO ---")
print("Todos los archivos han sido procesados. Revisa la carpeta de resultados.")

Importando librerías necesarias...
Librerías importadas correctamente. ✅

Cargando capas de división geopolítica...
Capas geopolíticas cargadas exitosamente. 🗺️

--- INICIANDO PROCESO DE ANÁLISIS ESPACIAL ---

Procesando Área de Estudio: 16_abril

   Procesando cobertura del año: 2018
     1/5: Cortando capa de uso de suelo...
     2/5: Calculando área en hectáreas...
     3/5: Asignando información geopolítica...
     4/5: Estandarizando tabla de atributos...
     5/5: Guardando resultado en: cob2018_16_abril.shp
     -> Archivo guardado con éxito. ✨

   Procesando cobertura del año: 2020
     1/5: Cortando capa de uso de suelo...
     2/5: Calculando área en hectáreas...
     3/5: Asignando información geopolítica...
     4/5: Estandarizando tabla de atributos...
     5/5: Guardando resultado en: cob2020_16_abril.shp
     -> Archivo guardado con éxito. ✨

   Procesando cobertura del año: 2022


c:\Users\nacho\miniforge3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


     1/5: Cortando capa de uso de suelo...
     2/5: Calculando área en hectáreas...
     3/5: Asignando información geopolítica...
     4/5: Estandarizando tabla de atributos...
     5/5: Guardando resultado en: cob2022_16_abril.shp
     -> Archivo guardado con éxito. ✨

Procesando Área de Estudio: 24_mayo

   Procesando cobertura del año: 2018


c:\Users\nacho\miniforge3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


     1/5: Cortando capa de uso de suelo...
     2/5: Calculando área en hectáreas...
     3/5: Asignando información geopolítica...
     4/5: Estandarizando tabla de atributos...
     5/5: Guardando resultado en: cob2018_24_mayo.shp
     -> Archivo guardado con éxito. ✨

   Procesando cobertura del año: 2020
     1/5: Cortando capa de uso de suelo...
     2/5: Calculando área en hectáreas...
     3/5: Asignando información geopolítica...
     4/5: Estandarizando tabla de atributos...
     5/5: Guardando resultado en: cob2020_24_mayo.shp
     -> Archivo guardado con éxito. ✨

   Procesando cobertura del año: 2022


...

c:\Users\nacho\miniforge3\envs\geo_env\Lib\site-packages\geopandas\tools\overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 8 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)
c:\Users\nacho\miniforge3\envs\geo_env\Lib\site-packages\geopandas\tools\overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 12 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)
c:\Users\nacho\miniforge3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

     4/5: Estandarizando tabla de atributos...
     5/5: Guardando resultado en: cob2022_tomebamba.shp
     -> Archivo guardado con éxito. ✨

Procesando Área de Estudio: voluntad_dios

   Procesando cobertura del año: 2018
     1/5: Cortando capa de uso de suelo...
     2/5: Calculando área en hectáreas...
     3/5: Asignando información geopolítica...
     4/5: Estandarizando tabla de atributos...
     5/5: Guardando resultado en: cob2018_voluntad_dios.shp
     -> Archivo guardado con éxito. ✨

   Procesando cobertura del año: 2020
     1/5: Cortando capa de uso de suelo...
     2/5: Calculando área en hectáreas...
     3/5: Asignando información geopolítica...
     4/5: Estandarizando tabla de atributos...
     5/5: Guardando resultado en: cob2020_voluntad_dios.shp
     -> Archivo guardado con éxito. ✨

   Procesando cobertura del año: 2022


c:\Users\nacho\miniforge3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


     1/5: Cortando capa de uso de suelo...
     2/5: Calculando área en hectáreas...
     3/5: Asignando información geopolítica...
     4/5: Estandarizando tabla de atributos...
     5/5: Guardando resultado en: cob2022_voluntad_dios.shp
     -> Archivo guardado con éxito. ✨

--- PROCESO FINALIZADO ---
Todos los archivos han sido procesados. Revisa la carpeta de resultados.


c:\Users\nacho\miniforge3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


# Análisis de deforestación

In [3]:
# ==============================================================================
# SCRIPT PARA ANÁLISIS DE DEFORESTACIÓN POR ÁREA DE ESTUDIO (EJECUCIÓN LOCAL)
# ==============================================================================
# Este script realiza los siguientes pasos:
# 1. Itera sobre cada shapefile de área de estudio en una carpeta específica.
# 2. Para cada área, recorta tres capas de deforestación (2018, 2020, 2022).
# 3. Realiza una operación 'dissolve' para unir todos los polígonos de
#    deforestación recortados en una sola geometría.
# 4. Calcula el área total de la deforestación en hectáreas (ha).
# 5. Guarda el resultado en un nuevo shapefile con un nombre estandarizado,
#    eliminando la terminación "_influencia" del nombre del área de estudio.
# ==============================================================================

import geopandas as gpd
import os
import warnings

# Ignorar advertencias futuras de geopandas para una salida más limpia.
warnings.simplefilter(action='ignore', category=FutureWarning)

# ==============================================================================
# 1. DEFINE TUS RUTAS AQUÍ
# ==============================================================================
# ¡IMPORTANTE! Reemplaza el texto entre comillas con las rutas a tus carpetas.
# Usa el formato r"C:\Usuarios\TuUsuario\..." para evitar errores en Windows.

# Carpeta que contiene los shapefiles de tus áreas de estudio.
ruta_areas_estudio = r"C:\Users\nacho\Documents\Github\GIZ\area_estudio"

# Carpeta donde se guardarán los resultados.
# Si no existe, el script la creará por ti.
ruta_resultados = r"C:\Users\nacho\Documents\Github\GIZ\def\corte"

# --- Rutas a tus capas de deforestación ---
# Define aquí la ruta completa a cada uno de tus archivos shapefile.
deforestacion_info = {
    '2018': r"C:\Users\nacho\Documents\Github\GIZ\def\v_fc010_deforestacion_16_18_aPolygon.shp",
    '2020': r"C:\Users\nacho\Documents\Github\GIZ\def\v_fc010_deforestacion_18_20_aPolygon.shp",
    '2022': r"C:\Users\nacho\Documents\Github\GIZ\def\v_fc010_deforestacion_20_22_aPolygon.shp"
}
# ==============================================================================

# Crear la carpeta de resultados si no existe.
if not os.path.exists(ruta_resultados):
    os.makedirs(ruta_resultados)
    print(f"Carpeta de resultados creada en: {ruta_resultados}")

# ==============================================================================
# 2. PROCESAMIENTO PRINCIPAL
# ==============================================================================
print("\n--- INICIANDO ANÁLISIS DE DEFORESTACIÓN ---")

# Validar que la carpeta de áreas de estudio exista.
if not os.path.isdir(ruta_areas_estudio):
    print(f"¡ERROR! La carpeta de áreas de estudio no se encontró en: {ruta_areas_estudio}")
    exit()

# Iterar sobre todos los archivos en la carpeta de áreas de estudio.
for area_shp_nombre in os.listdir(ruta_areas_estudio):
    if area_shp_nombre.endswith('.shp'):
        
        # Procesamiento del nombre del archivo de salida.
        nombre_base_area = os.path.splitext(area_shp_nombre)[0]
        # Eliminar la parte "_influencia" del nombre.
        nombre_salida_base = nombre_base_area.replace('_influencia', '')
        
        ruta_area_completa = os.path.join(ruta_areas_estudio, area_shp_nombre)

        print(f"\n==================================================")
        print(f"Procesando Área de Estudio: {nombre_base_area}")
        print(f"==================================================")

        try:
            # Cargar el shapefile del área de estudio.
            area_gdf = gpd.read_file(ruta_area_completa)
        except Exception as e:
            print(f"  -> Error al leer el archivo {area_shp_nombre}: {e}")
            continue # Salta al siguiente archivo si hay un error.

        # Iterar sobre cada capa de deforestación definida arriba.
        for año, ruta_def_completa in deforestacion_info.items():
            
            print(f"\n  Procesando capa de deforestación del año: {año}")

            # Verificar si el archivo de deforestación existe.
            if not os.path.exists(ruta_def_completa):
                print(f"  -> ¡ADVERTENCIA! No se encontró el archivo: {ruta_def_completa}. Saltando...")
                continue

            try:
                # Cargar la capa de deforestación.
                deforestacion_gdf = gpd.read_file(ruta_def_completa)

                # 1. VERIFICACIÓN DE CRS (Sistema de Coordenadas)
                # Es crucial que ambos GeoDataFrames tengan el mismo CRS.
                if area_gdf.crs != deforestacion_gdf.crs:
                    print(f"    -> Advertencia: Los CRS no coinciden. Reproyectando área de estudio al CRS de la capa de deforestación ({deforestacion_gdf.crs.to_string()})")
                    area_gdf = area_gdf.to_crs(deforestacion_gdf.crs)

                # 2. RECORTE (CLIP)
                print("    1/4: Recortando la capa de deforestación...")
                recorte_gdf = gpd.clip(deforestacion_gdf, area_gdf)

                # Verificar si el resultado del recorte está vacío.
                if recorte_gdf.empty:
                    print("    -> El área de estudio no se superpone con la capa de deforestación. No se generará archivo.")
                    continue

                # 3. DISOLVER (DISSOLVE)
                # Une todos los polígonos recortados en una sola geometría.
                print("    2/4: Uniendo polígonos (dissolve)...")
                # Creamos una columna temporal con un valor constante para poder disolver todo en uno.
                recorte_gdf['dissolve_id'] = 1
                disuelto_gdf = recorte_gdf.dissolve(by='dissolve_id')

                # 4. CÁLCULO DE ÁREA EN HECTÁREAS
                print("    3/4: Calculando área en hectáreas...")
                # El resultado de dissolve es un GeoSeries, lo convertimos a GeoDataFrame.
                disuelto_gdf = gpd.GeoDataFrame(geometry=disuelto_gdf.geometry)
                disuelto_gdf['area_ha'] = disuelto_gdf.geometry.area / 10000
                
                # 5. GUARDADO DEL RESULTADO
                nombre_salida = f"def{año[2:]}_{nombre_salida_base}.shp"
                ruta_salida_completa = os.path.join(ruta_resultados, nombre_salida)
                print(f"    4/4: Guardando resultado en: {nombre_salida}")

                disuelto_gdf.to_file(ruta_salida_completa, driver='ESRI Shapefile', encoding='utf-8')
                print(f"    -> Archivo guardado con éxito. ✨")

            except Exception as e:
                print(f"  -> Ocurrió un error inesperado al procesar la capa del año {año}: {e}")

print("\n--- PROCESO FINALIZADO ---")
print("Todos los archivos han sido procesados. Revisa la carpeta de resultados.")


--- INICIANDO ANÁLISIS DE DEFORESTACIÓN ---

Procesando Área de Estudio: 16_abril

  Procesando capa de deforestación del año: 2018
    1/4: Recortando la capa de deforestación...
    2/4: Uniendo polígonos (dissolve)...
    3/4: Calculando área en hectáreas...
    4/4: Guardando resultado en: def18_16_abril.shp
    -> Archivo guardado con éxito. ✨

  Procesando capa de deforestación del año: 2020


C:\Users\nacho\AppData\Local\Temp\ipykernel_18820\3899044288.py:127: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  disuelto_gdf.to_file(ruta_salida_completa, driver='ESRI Shapefile', encoding='utf-8')
c:\Users\nacho\miniforge3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'dissolve_id' to 'dissolve_i'
  ogr_write(


    1/4: Recortando la capa de deforestación...
    2/4: Uniendo polígonos (dissolve)...
    3/4: Calculando área en hectáreas...
    4/4: Guardando resultado en: def20_16_abril.shp
    -> Archivo guardado con éxito. ✨

  Procesando capa de deforestación del año: 2022


C:\Users\nacho\AppData\Local\Temp\ipykernel_18820\3899044288.py:127: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  disuelto_gdf.to_file(ruta_salida_completa, driver='ESRI Shapefile', encoding='utf-8')
c:\Users\nacho\miniforge3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'dissolve_id' to 'dissolve_i'
  ogr_write(


    1/4: Recortando la capa de deforestación...
    2/4: Uniendo polígonos (dissolve)...
    3/4: Calculando área en hectáreas...
    4/4: Guardando resultado en: def22_16_abril.shp
    -> Archivo guardado con éxito. ✨

Procesando Área de Estudio: 24_mayo

  Procesando capa de deforestación del año: 2018


...

C:\Users\nacho\AppData\Local\Temp\ipykernel_18820\3899044288.py:127: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  disuelto_gdf.to_file(ruta_salida_completa, driver='ESRI Shapefile', encoding='utf-8')
c:\Users\nacho\miniforge3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'dissolve_id' to 'dissolve_i'
  ogr_write(


    1/4: Recortando la capa de deforestación...
    2/4: Uniendo polígonos (dissolve)...
    3/4: Calculando área en hectáreas...
    4/4: Guardando resultado en: def20_voluntad_dios.shp
    -> Archivo guardado con éxito. ✨

  Procesando capa de deforestación del año: 2022


C:\Users\nacho\AppData\Local\Temp\ipykernel_18820\3899044288.py:127: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  disuelto_gdf.to_file(ruta_salida_completa, driver='ESRI Shapefile', encoding='utf-8')
c:\Users\nacho\miniforge3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'dissolve_id' to 'dissolve_i'
  ogr_write(


    1/4: Recortando la capa de deforestación...
    2/4: Uniendo polígonos (dissolve)...
    3/4: Calculando área en hectáreas...
    4/4: Guardando resultado en: def22_voluntad_dios.shp
    -> Archivo guardado con éxito. ✨

--- PROCESO FINALIZADO ---
Todos los archivos han sido procesados. Revisa la carpeta de resultados.


C:\Users\nacho\AppData\Local\Temp\ipykernel_18820\3899044288.py:127: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  disuelto_gdf.to_file(ruta_salida_completa, driver='ESRI Shapefile', encoding='utf-8')
c:\Users\nacho\miniforge3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'dissolve_id' to 'dissolve_i'
  ogr_write(


# Exportar datos a matiz de excel

In [11]:
# ==============================================================================
# SCRIPT PARA CONSOLIDAR DATOS DE SHAPEFILES EN UNA MATRIZ DE EXCEL
# ==============================================================================
# Este script realiza los siguientes pasos:
# 1. Lee todos los archivos .shp de una carpeta de entrada.
# 2. Para cada archivo, extrae los datos de su tabla de atributos.
# 3. Extrae el "AÑO" y el "NOMBRE" del área de estudio a partir del
#    nombre del archivo.
# 4. Consolida los datos de todos los archivos en una única tabla.
# 5. Renombra, ordena y convierte a MAYÚSCULAS todas las columnas de texto.
# 6. Guarda la tabla final como un único archivo de Excel (.xlsx).
# ==============================================================================

import geopandas as gpd
import pandas as pd
import os

# ==============================================================================
# 1. DEFINE TUS RUTAS AQUÍ
# ==============================================================================
# ¡IMPORTANTE! Reemplaza el texto entre comillas con tus rutas.
# Usa el formato r"C:\Ruta\Completa\..." para Windows.

# Carpeta que contiene los shapefiles de uso de suelo (los resultados del script anterior).
ruta_carpeta_shapefiles = r"C:\Users\nacho\Documents\Github\GIZ\results"

# Ruta completa donde se guardará el archivo Excel, incluyendo el nombre del archivo.
ruta_salida_excel = r"C:\Users\nacho\Documents\Github\GIZ\informe\matriz_v1.xlsx"
# ==============================================================================

# Lista para almacenar los DataFrames de cada shapefile procesado.
lista_de_datos = []

print("--- INICIANDO PROCESO DE CONSOLIDACIÓN ---")

# Validar que la carpeta de entrada exista.
if not os.path.isdir(ruta_carpeta_shapefiles):
    print(f"¡ERROR! La carpeta de entrada no se encontró en: {ruta_carpeta_shapefiles}")
    exit()

# Iterar sobre todos los archivos en la carpeta de entrada.
for nombre_archivo in os.listdir(ruta_carpeta_shapefiles):
    if nombre_archivo.endswith('.shp'):
        print(f"Procesando: {nombre_archivo}...")
        
        try:
            # --- Extracción de AÑO y NOMBRE desde el nombre del archivo ---
            # Ejemplo: "cob2018_6_junio_influencia.shp"
            
            # 1. Quitar el prefijo 'cob' y el sufijo '_influencia.shp'
            parte_central = nombre_archivo.replace('cob', '').replace('.shp', '')
            
            # 2. El año son los primeros 4 caracteres.
            año = parte_central[:4]
            
            # 3. El nombre es el resto, eliminando el primer guion bajo.
            nombre_area = parte_central[5:]
            
            # 4. Formatear el nombre (reemplazar '_' por ' ').
            nombre_area_formateado = nombre_area.replace('_', ' ',)

            # --- Carga y procesamiento de datos del shapefile ---
            ruta_completa_shp = os.path.join(ruta_carpeta_shapefiles, nombre_archivo)
            
            # Cargar el shapefile.
            gdf = gpd.read_file(ruta_completa_shp)
            
            # Si el shapefile está vacío, saltar al siguiente.
            if gdf.empty:
                print("  -> Advertencia: El shapefile está vacío. Saltando.")
                continue
            
            # Convertir a un DataFrame de pandas (ya no necesitamos la geometría).
            df = pd.DataFrame(gdf.drop(columns='geometry'))
            
            # Agregar las columnas extraídas del nombre del archivo.
            df['AÑO'] = año
            df['NOMBRE'] = nombre_area_formateado
            
            # Añadir el DataFrame procesado a nuestra lista.
            lista_de_datos.append(df)

        except Exception as e:
            print(f"  -> Error al procesar el archivo '{nombre_archivo}': {e}")
            print("  -> Es posible que el nombre del archivo no siga el patrón esperado.")
            continue

# --- Consolidación y guardado del archivo final ---
if not lista_de_datos:
    print("\nNo se encontraron datos para procesar. Asegúrate de que la carpeta de entrada contenga shapefiles válidos.")
else:
    print("\nConsolidando todos los datos...")
    
    # Unir todos los DataFrames de la lista en uno solo.
    matriz_final = pd.concat(lista_de_datos, ignore_index=True)
    
    # Renombrar las columnas para que coincidan con el formato final deseado.
    matriz_final = matriz_final.rename(columns={
        'provincia': 'PROVINCIA',
        'canton': 'CANTON',
        'parroquia': 'PARROQUIA',
        'uso_suelo': 'CATEGORÍA'
        # 'ha' ya tiene el nombre correcto.
    })
    
    # Seleccionar y reordenar las columnas en el orden final.
    columnas_finales = ['PROVINCIA', 'CANTON', 'PARROQUIA', 'NOMBRE', 'AÑO', 'CATEGORÍA', 'ha']
    matriz_final = matriz_final[columnas_finales]
    
    # === CAMBIO IMPORTANTE: CONVERTIR TODAS LAS COLUMNAS DE TEXTO A MAYÚSCULAS ===
    print("Convirtiendo texto a mayúsculas...")
    columnas_texto = ['PROVINCIA', 'CANTON', 'PARROQUIA', 'NOMBRE', 'CATEGORÍA']
    for col in columnas_texto:
        # Usamos .astype(str) para evitar errores si hay algún dato numérico perdido
        matriz_final[col] = matriz_final[col].astype(str).str.upper()
    
    # Guardar el DataFrame final en un archivo Excel.
    # 'index=False' evita que se guarde una columna extra con el índice del DataFrame.
    try:
        matriz_final.to_excel(ruta_salida_excel, index=False)
        print(f"\n--- PROCESO COMPLETADO ---")
        print(f"Matriz guardada exitosamente en: {ruta_salida_excel} ✅")
    except Exception as e:
        print(f"\n¡ERROR! No se pudo guardar el archivo Excel: {e}")

--- INICIANDO PROCESO DE CONSOLIDACIÓN ---
Procesando: cob2018_16_abril.shp...
Procesando: cob2018_24_mayo.shp...
Procesando: cob2018_28_marzo.shp...
Procesando: cob2018_29_marzo.shp...
Procesando: cob2018_6_junio.shp...
Procesando: cob2018_9_de_marzo.shp...
Procesando: cob2018_alto_huino.shp...
Procesando: cob2018_amazonas.shp...
Procesando: cob2018_angeles.shp...
Procesando: cob2018_arapino.shp...
Procesando: cob2018_aromacacao_a.shp...
Procesando: cob2018_aromacacao_b.shp...
Procesando: cob2018_aromacacao_c.shp...
Procesando: cob2018_atahualpa.shp...
Procesando: cob2018_calapucha.shp...
Procesando: cob2018_chimboracense.shp...
Procesando: cob2018_ejcf.shp...
Procesando: cob2018_el_inca.shp...
Procesando: cob2018_fano.shp...
Procesando: cob2018_florida.shp...
Procesando: cob2018_las_palmas.shp...
Procesando: cob2018_marianita_miguel.shp...
Procesando: cob2018_parcelas_7_julio.shp...
Procesando: cob2018_parcelas_shushufindi.shp...
Procesando: cob2018_parcelas_spcofanes.shp...
Procesan

In [12]:
# ==============================================================================
# SCRIPT PARA AÑADIR DATOS DE DEFORESTACIÓN A UNA MATRIZ DE EXCEL EXISTENTE
# ==============================================================================
# Este script realiza los siguientes pasos:
# 1. Lee una matriz de datos ya existente desde un archivo de Excel.
# 2. Crea una tabla de referencia (provincia, cantón, parroquia) a partir
#    de los datos de ese Excel.
# 3. Itera sobre una carpeta que contiene shapefiles de deforestación.
# 4. Para cada shapefile, extrae AÑO, NOMBRE y las HECTÁREAS de la columna "area_ha".
# 5. Asigna la información de provincia, cantón y parroquia usando la tabla
#    de referencia.
# 6. Agrega los datos de deforestación a la matriz original.
# 7. Guarda el resultado final en un nuevo archivo de Excel con la estructura final.
# ==============================================================================

import pandas as pd
import geopandas as gpd
import os

# ==============================================================================
# 1. DEFINE TUS RUTAS AQUÍ
# ==============================================================================
# ¡IMPORTANTE! Reemplaza el texto entre comillas con tus rutas.

# Ruta al archivo Excel que ya contiene los datos de uso de suelo.
ruta_excel_base = r"C:\Users\nacho\Documents\Github\GIZ\informe\matriz_v1.xlsx"

# Carpeta que contiene los shapefiles de deforestación (ej: def18_6_junio.shp).
ruta_carpeta_deforestacion = r"C:\Users\nacho\Documents\Github\GIZ\def\corte"

# Ruta de salida para el nuevo archivo Excel que contendrá TODOS los datos.
ruta_salida_excel_final = r"C:\Users\nacho\Documents\Github\GIZ\informe\matriz_v2.xlsx"
# ==============================================================================

# --- PASO 1: Cargar la matriz de Excel base ---
print(f"Cargando la matriz base desde: {ruta_excel_base}")
try:
    df_base = pd.read_excel(ruta_excel_base)
except FileNotFoundError:
    print(f"¡ERROR! No se encontró el archivo Excel en la ruta especificada. Revisa la ruta_excel_base.")
    exit()

# --- PASO 2: Crear la tabla de referencia geopolítica ---
print("Creando tabla de referencia para provincia, cantón y parroquia...")
lookup_geo = df_base[['NOMBRE', 'PROVINCIA', 'CANTON', 'PARROQUIA']].drop_duplicates(subset=['NOMBRE']).reset_index(drop=True)
print("Tabla de referencia creada con éxito.")

# --- PASO 3: Procesar los shapefiles de deforestación ---
print(f"\nProcesando shapefiles de la carpeta: {ruta_carpeta_deforestacion}")
if not os.path.isdir(ruta_carpeta_deforestacion):
    print(f"¡ERROR! No se encontró la carpeta de deforestación. Revisa la ruta_carpeta_deforestacion.")
    exit()

lista_datos_deforestacion = []

for nombre_archivo in os.listdir(ruta_carpeta_deforestacion):
    if nombre_archivo.endswith('.shp'):
        print(f"  -> Procesando: {nombre_archivo}")
        try:
            # Extraer AÑO y NOMBRE del nombre del archivo (ej: "def18_6_junio.shp")
            año_corto = nombre_archivo[3:5]
            año = f"20{año_corto}"
            
            nombre_base = os.path.splitext(nombre_archivo)[0][6:]
            nombre_area = nombre_base.replace('_', ' ').upper()
            
            # Cargar el shapefile
            ruta_completa_shp = os.path.join(ruta_carpeta_deforestacion, nombre_archivo)
            gdf_defo = gpd.read_file(ruta_completa_shp)
            
            # --- CORRECCIÓN: Leer específicamente de la columna 'area_ha' ---
            if 'area_ha' in gdf_defo.columns:
                hectareas = gdf_defo['area_ha'].iloc[0]
            else:
                print(f"    -> Advertencia: No se encontró la columna 'area_ha' en {nombre_archivo}. Se asignará 0.")
                hectareas = 0

            # Crear un diccionario con los nuevos datos, usando 'ha' como clave final
            nuevo_registro = {
                'NOMBRE': nombre_area,
                'AÑO': int(año),
                'CATEGORÍA': 'DEFORESTACION',
                'ha': hectareas  # La clave aquí es 'ha' para que coincida con la matriz final
            }
            lista_datos_deforestacion.append(nuevo_registro)

        except Exception as e:
            print(f"    -> Error al procesar el archivo '{nombre_archivo}': {e}")

# --- PASO 4: Combinar los datos ---
print("\nAñadiendo datos de deforestación a la matriz...")
df_deforestacion = pd.DataFrame(lista_datos_deforestacion)

# Usar la tabla de referencia para añadir la información geopolítica
df_deforestacion_completo = pd.merge(df_deforestacion, lookup_geo, on='NOMBRE', how='left')

# Unir el DataFrame base con los nuevos datos de deforestación
df_final = pd.concat([df_base, df_deforestacion_completo], ignore_index=True)

# Asegurar el orden y nombre final de las columnas
columnas_finales = ['PROVINCIA', 'CANTON', 'PARROQUIA', 'NOMBRE', 'AÑO', 'CATEGORÍA', 'ha']
# El nombre de la columna CATEGORIA en la matriz base ya viene del script anterior.
# Lo renombramos por si acaso viniera con otro nombre.
df_final = df_final.rename(columns={'CATEGORIA': 'CATEGORÍA'})
df_final = df_final[columnas_finales]

# Asegurarse de que todas las columnas de texto estén en mayúsculas
columnas_texto = ['PROVINCIA', 'CANTON', 'PARROQUIA', 'NOMBRE', 'CATEGORÍA']
for col in columnas_texto:
    if col in df_final.columns:
        df_final[col] = df_final[col].astype(str).str.upper()

# --- PASO 5: Guardar el resultado final ---
try:
    df_final.to_excel(ruta_salida_excel_final, index=False)
    print(f"\n--- PROCESO COMPLETADO ---")
    print(f"Matriz final con datos de deforestación guardada en: {ruta_salida_excel_final} ✅")
except Exception as e:
    print(f"\n¡ERROR! No se pudo guardar el archivo Excel final: {e}")

Cargando la matriz base desde: C:\Users\nacho\Documents\Github\GIZ\informe\matriz_v1.xlsx
Creando tabla de referencia para provincia, cantón y parroquia...
Tabla de referencia creada con éxito.

Procesando shapefiles de la carpeta: C:\Users\nacho\Documents\Github\GIZ\def\corte
  -> Procesando: def18_16_abril.shp
  -> Procesando: def18_24_mayo.shp
  -> Procesando: def18_28_marzo.shp
  -> Procesando: def18_29_marzo.shp
  -> Procesando: def18_6_junio.shp
  -> Procesando: def18_9_de_marzo.shp
  -> Procesando: def18_alto_huino.shp
  -> Procesando: def18_amazonas.shp
  -> Procesando: def18_angeles.shp
  -> Procesando: def18_arapino.shp
  -> Procesando: def18_aromacacao_a.shp
  -> Procesando: def18_aromacacao_b.shp
  -> Procesando: def18_aromacacao_c.shp
  -> Procesando: def18_atahualpa.shp
  -> Procesando: def18_chimboracense.shp
  -> Procesando: def18_ejcf.shp
  -> Procesando: def18_el_inca.shp
  -> Procesando: def18_fano.shp
  -> Procesando: def18_florida.shp
  -> Procesando: def18_mariani

In [13]:
# ==============================================================================
# SCRIPT PARA AÑADIR EL ÁREA REFERENCIAL A LA MATRIZ DE EXCEL
# ==============================================================================
# Este script realiza los siguientes pasos:
# 1. Itera sobre una carpeta con los shapefiles de las áreas de estudio originales.
# 2. Para cada shapefile, calcula su área total en hectáreas.
# 3. Almacena estas áreas en una tabla de referencia asociadas al nombre del área.
# 4. Carga la matriz de Excel existente.
# 5. Crea una nueva columna "ha_REFERENCIAL" en la matriz, llenándola con
#    el área correspondiente a cada "NOMBRE" de área de estudio.
# 6. Guarda el resultado en un nuevo archivo de Excel.
# ==============================================================================

import pandas as pd
import geopandas as gpd
import os

# ==============================================================================
# 1. DEFINE TUS RUTAS AQUÍ
# ==============================================================================
# ¡IMPORTANTE! Reemplaza el texto entre comillas con tus rutas.

# Carpeta con los SHAPEFILES ORIGINALES de las áreas de estudio.
# (ej: "6_junio_influencia.shp")
ruta_areas_estudio = r"C:\Users\nacho\Documents\Github\GIZ\area_estudio"

# Ruta al archivo Excel que se va a actualizar (el resultado del script anterior).
ruta_excel_matriz = r"C:\Users\nacho\Documents\Github\GIZ\informe\matriz_v2.xlsx"

# Ruta de salida para el nuevo archivo Excel con la columna añadida.
ruta_excel_salida = r"C:\Users\nacho\Documents\Github\GIZ\informe\matriz_v3.xlsx"
# ==============================================================================

# --- PASO 1: Calcular el área de cada shapefile y crear una tabla de referencia ---
print(f"Calculando áreas desde la carpeta: {ruta_areas_estudio}")

# Diccionario para guardar las áreas: {'NOMBRE_AREA': area_en_hectareas}
areas_referenciales = {}

if not os.path.isdir(ruta_areas_estudio):
    print(f"¡ERROR! No se encontró la carpeta de áreas de estudio. Revisa la ruta.")
    exit()

for nombre_archivo in os.listdir(ruta_areas_estudio):
    if nombre_archivo.endswith('.shp'):
        try:
            # Formatear el nombre del archivo para que coincida con la columna "NOMBRE"
            # ej: "6_junio_influencia.shp" -> "6 JUNIO"
            nombre_base = os.path.splitext(nombre_archivo)[0]
            nombre_formateado = nombre_base.replace('_influencia', '').replace('_', ' ').upper()

            # Cargar el shapefile
            ruta_completa_shp = os.path.join(ruta_areas_estudio, nombre_archivo)
            gdf = gpd.read_file(ruta_completa_shp)
            
            # Disolver todos los polígonos en uno solo para asegurar un cálculo de área total
            gdf_disuelto = gdf.dissolve()
            
            # Calcular el área en hectáreas (área en m² / 10000)
            # .iloc[0] se usa para obtener el valor del primer (y único) polígono disuelto
            area_ha = gdf_disuelto.geometry.area.iloc[0] / 10000
            
            # Guardar en el diccionario
            areas_referenciales[nombre_formateado] = area_ha
            print(f"  -> Área '{nombre_formateado}': {area_ha:.2f} ha")

        except Exception as e:
            print(f"    -> Error al procesar el archivo '{nombre_archivo}': {e}")


# --- PASO 2: Cargar la matriz de Excel y añadir la nueva columna ---
print(f"\nCargando la matriz base desde: {ruta_excel_matriz}")
try:
    df_matriz = pd.read_excel(ruta_excel_matriz)
except FileNotFoundError:
    print(f"¡ERROR! No se encontró el archivo Excel en la ruta especificada.")
    exit()

print("Añadiendo la nueva columna 'ha_REFERENCIAL'...")

# Usar el método .map() para crear la nueva columna.
# Este método usa el diccionario 'areas_referenciales' para asignar el área
# correcta a cada fila basándose en el valor de la columna "NOMBRE".
df_matriz['ha_REFERENCIAL'] = df_matriz['NOMBRE'].map(areas_referenciales)

# Rellenar con 0 si algún nombre no se encontró (opcional, pero buena práctica)
df_matriz['ha_REFERENCIAL'].fillna(0, inplace=True)


# --- PASO 3: Guardar el resultado final ---
try:
    df_matriz.to_excel(ruta_excel_salida, index=False)
    print(f"\n--- PROCESO COMPLETADO ---")
    print(f"Matriz final con área referencial guardada en: {ruta_excel_salida} ✅")
except Exception as e:
    print(f"\n¡ERROR! No se pudo guardar el archivo Excel final: {e}")

Calculando áreas desde la carpeta: C:\Users\nacho\Documents\Github\GIZ\area_estudio
  -> Área '16 ABRIL': 1027.35 ha
  -> Área '24 MAYO': 1096.33 ha
  -> Área '28 MARZO': 1090.10 ha
  -> Área '29 MARZO': 1368.73 ha
  -> Área '6 JUNIO': 6017.63 ha
  -> Área '9 DE MARZO': 141.78 ha
  -> Área 'ALTO HUINO': 1228.43 ha
  -> Área 'AMAZONAS': 729.42 ha
  -> Área 'ANGELES': 1724.60 ha
  -> Área 'ARAPINO': 4108.17 ha
  -> Área 'AROMACACAO A': 1763.83 ha
  -> Área 'AROMACACAO B': 3226.20 ha
  -> Área 'AROMACACAO C': 12241.54 ha
  -> Área 'ATAHUALPA': 895.08 ha
  -> Área 'CALAPUCHA': 198.94 ha
  -> Área 'CHIMBORACENSE': 784.87 ha
  -> Área 'EJCF': 5154.73 ha
  -> Área 'EL INCA': 2084.95 ha
  -> Área 'FANO': 770.65 ha
  -> Área 'FLORIDA': 1231.92 ha
  -> Área 'LAS PALMAS': 121.90 ha
  -> Área 'MARIANITA MIGUEL': 808.60 ha
  -> Área 'PARCELAS 7 JULIO': 2417.63 ha
  -> Área 'PARCELAS SHUSHUFINDI': 3996.63 ha
  -> Área 'PARCELAS SPCOFANES': 1739.02 ha
  -> Área 'RIOS VENCEDORES': 4306.67 ha
  -> Área

C:\Users\nacho\AppData\Local\Temp\ipykernel_18820\3259308868.py:61: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  area_ha = gdf_disuelto.geometry.area.iloc[0] / 10000
